In [1]:
import glob
import pandas as pd 

# Specify what folders do we need
INPUT_FOLDER      = './data_preprocessed4/'
COLUMNS = ['velocity', 'acceleration', 'trajectory_id', 'subfolder',
        'labels', 'v_ave', 'v_med', 'v_max', 'v_std', 'a_ave', 'a_med', 'a_max', 'a_std']

list_df_metadata = []

# Find all files that ends with .csv, read and append to the list_df_metadata
for filename in glob.glob(INPUT_FOLDER + '*.csv'):
    df_data = pd.read_csv(filename)
    list_df_metadata.append(df_data)
    
df_metadata = pd.concat(list_df_metadata)
df_metadata = df_metadata.drop(['Unnamed: 0'], axis=1)
df_metadata = df_metadata.reset_index(drop=True)
df_metadata.head()

/home/zh.shakenov/miniconda3/envs/disser/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,lat,long,altitude,datetime,timedelta,distance,velocity,acceleration,trajectory_id,subfolder,...,a_ave,a_med,a_max,a_min,a_std,a_rol,a_rsd,a_qu1,a_qu2,a_qu3
0,40.061035,116.407049,64.0,2011-05-29 22:30:19,0 days 00:00:01.000000000,13.357307,13.357307,-0.810261,20110529222904,168,...,-0.029584,0.0,8.873882,-14.649245,0.731106,-0.193232,3.278783,-0.531929,0.024086,0.678226
1,40.060915,116.407060,65.0,2011-05-29 22:30:20,0 days 00:00:01.000000000,12.547046,12.547046,0.810261,20110529222904,168,...,-0.029584,0.0,8.873882,-14.649245,0.731106,-0.107458,3.269948,-0.333394,0.050601,0.784147
2,40.060802,116.407060,63.0,2011-05-29 22:30:21,0 days 00:00:01.000000000,13.357307,13.357307,-0.634696,20110529222904,168,...,-0.029584,0.0,8.873882,-14.649245,0.731106,-0.118051,3.271165,-0.531929,0.050601,0.784147
3,40.060682,116.407071,63.0,2011-05-29 22:30:22,0 days 00:00:01.000000000,12.722611,12.722611,-1.113277,20110529222904,168,...,-0.029584,0.0,8.873882,-14.649245,0.731106,-0.118465,3.271294,-0.531929,0.050601,0.784147
4,40.060568,116.407086,62.0,2011-05-29 22:30:23,0 days 00:00:01.000000000,11.609334,11.609334,-1.358473,20110529222904,168,...,-0.029584,0.0,8.873882,-14.649245,0.731106,-0.152541,3.278956,-0.618968,0.050601,0.784147


In [2]:
X_COLUMNS = ['v_rol', 'v_rsd', 'v_qu1','v_qu2', 'v_qu3', 'a_rol', 'a_rsd', 'a_qu1', 'a_qu2', 'a_qu3']
Y_COLUMN  = ['labels']

In [3]:
print(df_metadata.shape)
# df_metadata2 = df_metadata.dropna(subset=['labels'], axis=0)
df_metadata2 = df_metadata.dropna()
print(df_metadata2.shape)
X = df_metadata2[X_COLUMNS]
Y = df_metadata2[Y_COLUMN]

(23380929, 31)
(4583070, 31)


In [4]:
df_metadata2['labels'].value_counts(dropna=False)

walk          1228399
bus           1158420
bike           705599
train          558909
car            484383
subway         234119
taxi           198446
airplane         8990
boat             3502
run              1965
motorcycle        338
Name: labels, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123, stratify=Y)

In [6]:
print(y_train['labels'].value_counts())
print(y_test['labels'].value_counts())

walk          859879
bus           810894
bike          493919
train         391236
car           339068
subway        163883
taxi          138912
airplane        6293
boat            2452
run             1376
motorcycle       237
Name: labels, dtype: int64
walk          368520
bus           347526
bike          211680
train         167673
car           145315
subway         70236
taxi           59534
airplane        2697
boat            1050
run              589
motorcycle       101
Name: labels, dtype: int64


In [7]:
import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [8]:
# use a full grid over all parameters
param_grid = {"max_depth": [None],
              "max_features": [3],
              "min_samples_split": [10],
              "min_samples_leaf": [1],
              "bootstrap": [False],
              "criterion": ["gini"]}

In [9]:
# Utility function to report best scores
def report(results, n_top=1):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [10]:
clf = RandomForestClassifier(n_estimators=20, n_jobs=-1)


# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_train, y_train.values.ravel())

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

GridSearchCV took 363.66 seconds for 1 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.822 (std: 0.000)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 10}



In [11]:
clf.fit(X_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
clf.fit_transform(X_train, y_train.values.ravel())

AttributeError: 'RandomForestClassifier' object has no attribute 'fit_transform'

In [14]:
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)